In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
device = 'cpu'
from textgcn import TextGCN

In [ ]:
def training(epoch,adj,features,labels,model,opti,loss_func,length1,length2,length3,file):
    training_loss=[]
    test_loss=[]
    training_accuracy=[]
    test_accuracy=[]
    macro_av_f1=[]
    for i in range(epoch):
        print("Epoch: ",i+1)
        model.train()
        opti.zero_grad()
        output=model(adj,features).to(device)
        output1=output[-(length1+length2+length3):-length2-length3]
        loss=loss_func(output1,labels[:-length2])
        loss.backward()
        opti.step()
        print("Loss: ",loss.item())
        training_loss.append(loss.item())
        accuracy(adj,features,labels,model,length1,length2,length3,loss_func,training_accuracy,test_accuracy,test_loss,macro_av_f1)
        torch.save(model,file+"model"+str(i+1)+".pt")

        print("\n")
    #Save the graph of training loss and test loss and accuracy and macro average f1 score
    plt.plot(training_loss)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss")
    plt.savefig(file+"training_loss.png")
    plt.close()

    plt.plot(test_loss)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Test Loss")
    plt.savefig(file+"test_loss.png")
    plt.close()

    plt.plot(training_accuracy)
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Training Accuracy")
    plt.savefig(file+"training_accuracy.png")
    plt.close()

    plt.plot(test_accuracy)
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Test Accuracy")
    plt.savefig(file+"test_accuracy.png")
    plt.close()

    plt.plot(macro_av_f1)
    plt.xlabel("Epoch")
    plt.ylabel("Macro Average F1 Score")
    plt.title("Macro Average F1 Score")
    plt.savefig(file+"macro_average_f1.png")
    plt.close()

    #Save all these values in a txt file
    f=open(file+"result.txt","w")
    for i in range(epoch):
        f.write("Epoch: "+str(i+1)+"\n")
        f.write("Training Loss: "+str(training_loss[i])+"\n")
        f.write("Test Loss: "+str(test_loss[i])+"\n")
        f.write("Training Accuracy: "+str(training_accuracy[i])+"\n")
        f.write("Test Accuracy: "+str(test_accuracy[i])+"\n")
        f.write("Macro Average F1 Score: "+str(macro_av_f1[i])+"\n")
        f.write("\n")
    f.close()
    
def accuracy(adj,features,labels,model,length1,length2,length3,loss_func,training_accuracy,test_accuracy,test_loss,macro_av_f1_arr):
    model.eval()
    with torch.no_grad():   
        output=model(adj,features).to(device)
        o1=output[-(length1+length2+length3):-length2-length3]
        o2=output[-length2-length3:-length3]
        l1=labels[:-length2]
        l2=labels[length1:]
        loss_test=loss_func(o2,l2)
        accu1=torch.sum(torch.argmax(o1,dim=1)==l1).item()/len(l1)
        accu2=torch.sum(torch.argmax(o2,dim=1)==l2).item()/len(l2)
        #Calculate macro average f1 score
        macro_av_f1=0
        for i in range(2):
            tp=0
            fp=0
            fn=0
            for j in range(len(l2)):
                if torch.argmax(o2,dim=1)[j]==i:
                    if l2[j]==i:
                        tp+=1
                    else:
                        fp+=1
                else:
                    if l2[j]==i:
                        fn+=1
            if tp+fp+fn==0:
                macro_av_f1+=0
            else:
                macro_av_f1+=2*tp/(2*tp+fp+fn)
        macro_av_f1/=2
        print("Accuracy for training: ",accu1)
        training_accuracy.append(accu1)
        print("Accuracy for test: ",accu2)
        test_accuracy.append(accu2)
        print("Loss for test: ",loss_test.item())
        test_loss.append(loss_test.item())
        print("Macro Average F1 Score: ",macro_av_f1)
        macro_av_f1_arr.append(macro_av_f1)



In [ ]:
adj=torch.load('../adj.pt')
labels=torch.load('../labels.pt')

In [ ]:
#Read length.txt
with open('../utils/length.txt','r') as f:
    length=f.read()
    length=length.split()
    length1=int(length[0])
    length2=int(length[1])
    length3=int(length[2])


In [ ]:
import os
a=64
b=16
dropout=0.5
features=torch.load("../features.pt")
name="modelfinal/0.01_"+str(a)+"_"+str(b)+"/"
adj=torch.load('../adj.pt')
model=TextGCN(len(adj),a,b,2,dropout).to(device)
opti=torch.optim.Adam(model.parameters(),lr=0.01)
loss_func=torch.nn.CrossEntropyLoss().to(device)
if not os.path.exists(name):
    os.makedirs(name)
training(250,adj,features,labels,model,opti,loss_func,length1,length2,length3,name)